# Data : https://www.kaggle.com/openaddresses/openaddresses-europe?select=germany.csv

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from fuzzywuzzy import fuzz # (you can install rapidfuzz for faster speed)
from fuzzywuzzy import process
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
#Loaidng the address data
addresses=pd.read_csv("austria.csv")

In [3]:
addresses.dtypes

LON         float64
LAT         float64
NUMBER       object
STREET       object
UNIT        float64
CITY         object
DISTRICT    float64
REGION      float64
POSTCODE      int64
ID          float64
HASH         object
dtype: object

In [4]:
addresses.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH
0,11.406601,47.236230,21,Bachgangweg,NaN,NaN,NaN,NaN,6080,NaN,6c0b77620f8da36b
1,11.409717,47.238259,51,Bachgangweg,NaN,NaN,NaN,NaN,6080,NaN,bc4a01439fb83ebc
2,11.401644,47.236334,1,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,ffdfe99ada1f0968
3,11.400591,47.236792,15,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,859e05ca87e751e2
4,11.400455,47.236940,21,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,5fc9979b6e4ce22d


In [5]:
#converting postalcode to string
addresses["POSTCODE"]=addresses["POSTCODE"].astype(str)

In [6]:
#creating a complete address column by combining Number,Street,CIty and Postcode
addresses["complete_address"]=addresses["NUMBER"]+" "+addresses["STREET"]+ " "+addresses["POSTCODE"]

In [7]:
addresses.head()

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH,complete_address
0,11.406601,47.236230,21,Bachgangweg,NaN,NaN,NaN,NaN,6080,NaN,6c0b77620f8da36b,21 Bachgangweg 6080
1,11.409717,47.238259,51,Bachgangweg,NaN,NaN,NaN,NaN,6080,NaN,bc4a01439fb83ebc,51 Bachgangweg 6080
2,11.401644,47.236334,1,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,ffdfe99ada1f0968,1 Burgweg 6080
3,11.400591,47.236792,15,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,859e05ca87e751e2,15 Burgweg 6080
4,11.400455,47.236940,21,Burgweg,NaN,NaN,NaN,NaN,6080,NaN,5fc9979b6e4ce22d,21 Burgweg 6080


In [8]:
addresses=addresses[:1000]

###  Usecase : You have two dataframes containing address column and you want to match the closest address

In [9]:
#Lets take a sample of 10 addresses from above data to match
sample = addresses.sample(n = 10)

In [10]:
sample

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH,complete_address
192,11.366928,47.267569,5,Achselkopfweg,NaN,NaN,NaN,NaN,6020,NaN,17355f72f81900d7,5 Achselkopfweg 6020
145,11.404654,47.236488,30,Robert-Stolz-Weg,NaN,NaN,NaN,NaN,6080,NaN,9d618fa60908d87d,30 Robert-Stolz-Weg 6080
25,11.404738,47.237768,20b,Grillhofweg,NaN,NaN,NaN,NaN,6080,NaN,8b56162b5ddc6773,20b Grillhofweg 6080
920,11.445805,47.267435,54,Archenweg,NaN,NaN,NaN,NaN,6020,NaN,4f55efe76c40f3a1,54 Archenweg 6020
827,11.390075,47.264134,27,Anichstraße,NaN,NaN,NaN,NaN,6020,NaN,79dac13f2b6164f5,27 Anichstraße 6020
848,11.395449,47.256011,11,Anton-Melzer-Straße,NaN,NaN,NaN,NaN,6020,NaN,ab64ac9ce95a365d,11 Anton-Melzer-Straße 6020
285,11.351971,47.269453,26,Allerheiligenhofweg,NaN,NaN,NaN,NaN,6020,NaN,cc8223a335ec48df,26 Allerheiligenhofweg 6020
732,11.391340,47.260753,16a,Andreas-Hofer-Straße,NaN,NaN,NaN,NaN,6020,NaN,b9579e153bae1081,16a Andreas-Hofer-Straße 6020
7,11.402263,47.236681,1,Grillhofweg,NaN,NaN,NaN,NaN,6080,NaN,1e3cb64b2eddd61f,1 Grillhofweg 6080
393,11.407633,47.263935,15,Amraser Straße,NaN,NaN,NaN,NaN,6020,NaN,d8b307178d27f496,15 Amraser Straße 6020


In [11]:
#Thresholds can be adjusted 
FULL_MATCHING_THRESHOLD = 85
PARTIAL_MATCHING_THRESHOLD = 85
SORT_MATCHING_THRESHOLD = 85
TOKEN_MATCHING_THRESHOLD = 85
MAX_MATCHES=1



def find_matches(matchThis):
    rows = list(addresses['complete_address'])
    matches= process.extractBests(matchThis,rows,scorer=fuzz.ratio,score_cutoff=FULL_MATCHING_THRESHOLD,limit=MAX_MATCHES)
    if len(matches)==0:
        matches= process.extractBests(matchThis,rows,scorer=fuzz.partial_ratio,score_cutoff=PARTIAL_MATCHING_THRESHOLD,limit=MAX_MATCHES);
        if len(matches)==0:
            matches= process.extractBests(matchThis,rows,scorer=fuzz.token_set_ratio,score_cutoff=TOKEN_MATCHING_THRESHOLD,limit=MAX_MATCHES);
            if len(matches)==0:
                matches= process.extractBests(matchThis,rows,scorer=fuzz.token_sort_ratio,score_cutoff=SORT_MATCHING_THRESHOLD,limit=MAX_MATCHES);
    
    return matches if len(matches)>0 else None


In [12]:
addresses['complete_address']=addresses['complete_address'].astype(str)

In [13]:
sample['top_match']=Parallel(n_jobs=63,backend='threading')(delayed(find_matches)(j) for j in tqdm(sample['complete_address'],total=len(sample['complete_address'])))    


100%|██████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


In [14]:
sample

,LON,LAT,NUMBER,STREET,UNIT,CITY,DISTRICT,REGION,POSTCODE,ID,HASH,complete_address,top_match
192,11.366928,47.267569,5,Achselkopfweg,NaN,NaN,NaN,NaN,6020,NaN,17355f72f81900d7,5 Achselkopfweg 6020,"[(5 Achselkopfweg 6020, 100)]"
145,11.404654,47.236488,30,Robert-Stolz-Weg,NaN,NaN,NaN,NaN,6080,NaN,9d618fa60908d87d,30 Robert-Stolz-Weg 6080,"[(30 Robert-Stolz-Weg 6080, 100)]"
25,11.404738,47.237768,20b,Grillhofweg,NaN,NaN,NaN,NaN,6080,NaN,8b56162b5ddc6773,20b Grillhofweg 6080,"[(20b Grillhofweg 6080, 100)]"
920,11.445805,47.267435,54,Archenweg,NaN,NaN,NaN,NaN,6020,NaN,4f55efe76c40f3a1,54 Archenweg 6020,"[(54 Archenweg 6020, 100)]"
827,11.390075,47.264134,27,Anichstraße,NaN,NaN,NaN,NaN,6020,NaN,79dac13f2b6164f5,27 Anichstraße 6020,"[(27 Anichstraße 6020, 100)]"
848,11.395449,47.256011,11,Anton-Melzer-Straße,NaN,NaN,NaN,NaN,6020,NaN,ab64ac9ce95a365d,11 Anton-Melzer-Straße 6020,"[(11 Anton-Melzer-Straße 6020, 100)]"
285,11.351971,47.269453,26,Allerheiligenhofweg,NaN,NaN,NaN,NaN,6020,NaN,cc8223a335ec48df,26 Allerheiligenhofweg 6020,"[(26 Allerheiligenhofweg 6020, 100)]"
732,11.391340,47.260753,16a,Andreas-Hofer-Straße,NaN,NaN,NaN,NaN,6020,NaN,b9579e153bae1081,16a Andreas-Hofer-Straße 6020,"[(16a Andreas-Hofer-Straße 6020, 100)]"
7,11.402263,47.236681,1,Grillhofweg,NaN,NaN,NaN,NaN,6080,NaN,1e3cb64b2eddd61f,1 Grillhofweg 6080,"[(1 Grillhofweg 6080, 100)]"
393,11.407633,47.263935,15,Amraser Straße,NaN,NaN,NaN,NaN,6020,NaN,d8b307178d27f496,15 Amraser Straße 6020,"[(15 Amraser Straße 6020, 100)]"


### Results and Conclusion :

As you can see the top match column has been created and it also shows the label of match (Since we are getting exact matches so the match label is 100)

In [15]:
# We can try searching a random address as below:
match=find_matches("15 Achselkopfweg 6020")

In [17]:
match # As you can see we are getting the nearest matches

[('1 Achselkopfweg 6020', 98)]